<a href="https://colab.research.google.com/github/41371103hjnh/Programming_Project/blob/main/Copy_%E7%A8%8B%E5%BC%8F%E8%AA%9E%E8%A8%80%E6%9C%9F%E6%9C%AB%E5%A0%B1%E5%91%8A%EF%BC%8D%E6%9C%80%E7%B5%82%E7%89%88.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
# -*- coding: utf-8 -*-
# Cell 1：安裝必要套件

!pip install -q gradio gspread google-auth google-auth-oauthlib google-auth-httplib2 google-generativeai pandas
!pip install -U google-generativeai

print("✅ 所有套件安裝完成。")

✅ 所有套件安裝完成。


In [7]:
# Cell 2：準備 lots.csv 檔案（優先讀取 Colab Files）

import pandas as pd
import io
import os

FILE_NAME = "lots.csv"

if os.path.exists(FILE_NAME):
    # ✅ 情況 1：Colab 左側 Files 已經有 lots.csv
    df = pd.read_csv(FILE_NAME, dtype=str).fillna("")
    print("✅ 已從 Colab Files 讀取 lots.csv")

else:
    # ⚠️ 情況 2：沒有檔案 → 建立最小可執行範例
    print("⚠️ Colab Files 未找到 lots.csv，建立範例檔案")

    csv_data = """lot_id,original,base_mean,source_url
1,世間萬物皆有緣，緣到自然成雙對。,"意指緣份天註定，時候到了自然會遇到對的人。",https://example.com/lot/1
2,花開花謝皆有時，莫為落花空悲切。,感情需要時間醞釀，不要因為一時的錯過而過度傷心。,https://example.com/lot/2
3,心誠則靈求神助，月老紅線兩頭牽。,強調真誠和尋求外力（如月老）的幫助，緣份終會到來。,https://example.com/lot/3
"""
    df = pd.read_csv(io.StringIO(csv_data), dtype=str).fillna("")
    df.to_csv(FILE_NAME, index=False, encoding="utf-8-sig")

    print("✅ 已建立範例 lots.csv（之後可上傳完整版覆蓋）")


✅ 已從 Colab Files 讀取 lots.csv


In [8]:
## -*- coding: utf-8 -*-
# Cell 3：參數設定、套件匯入與 API / Sheet / Gemini 連線（單一 Key）

import requests
import datetime
import json
import re
import pandas as pd
import random
import traceback
import io

import gradio as gr

from google.colab import auth, userdata
import gspread
from google.auth import default

import google.generativeai as genai
from google import genai as client_genai  # client-based 呼叫用

# ===============================
# ① RapidAPI Key（Love Calculator / Horostory）
# ===============================
RAPIDAPI_KEY = "ce4ce1e834msh20af08bc2460ec1p1466e7jsnffbfbe464b40"

# ===============================
# ② Google Sheet 設定
# ===============================
SHEET_URL = "https://docs.google.com/spreadsheets/d/106PoiJgG46Pc-wlPRBtCvibLpmrhF1kOCSND-QQRg6g/edit?gid=0#gid=0"
WORKSHEET_NAME = "緣分匹配"

# ===============================
# ③ Gemini API Key（Secret + fallback）
# ===============================
def get_gemini_key(secret_name="gemini_key", fallback=None):
    try:
        key = userdata.get(secret_name)
        if key:
            print(f"🔐 已從 Colab Secret 讀取 {secret_name}")
            return key
    except Exception as e:
        print(f"⚠️ 無法從 Colab Secret 讀取 {secret_name}：{e}")

    if fallback:
        print(f"⚠️ 改用 fallback key（{secret_name}）")
        return fallback

    raise RuntimeError("❌ 無法取得 Gemini API Key")

# 🔹 單一 Gemini Key
GEMINI_KEY = get_gemini_key(
    secret_name="p",                 # ← 你現在用的 Secret 名稱
    fallback="AIzaSyXXXXXXXXXXXXXXX" # ← 你的付費 Gemini API Key
)

# ===============================
# ④ 初始化 Gemini（單一 Key）
# ===============================
genai.configure(api_key=GEMINI_KEY)

MODEL_NAME = "models/gemini-2.5-flash"
gemini_model = genai.GenerativeModel(MODEL_NAME)

# client API（給月老籤詩用）
yuelao_client = client_genai.Client(api_key=GEMINI_KEY)

print("✅ Gemini API（單一 Key）初始化完成")

# ===============================
# ⑤ Google Colab 驗證 & 連接 Google Sheet
# ===============================
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

try:
    sh = gc.open_by_url(SHEET_URL)
    ws = sh.worksheet(WORKSHEET_NAME)
    print(f"✅ 已連線到試算表：{sh.title} → 工作表：{WORKSHEET_NAME}")
except Exception as e:
    print(f"❌ 找不到試算表或工作表：{e}（配對結果將不寫入）")
    ws = None


🔐 已從 Colab Secret 讀取 p
✅ Gemini API（單一 Key）初始化完成
✅ 已連線到試算表：緣來如此 → 工作表：緣分匹配


In [9]:
# -*- coding: utf-8 -*-
# Cell 4：核心功能函式定義 (Function Definitions)

# =================================================================
# 緣分匹配功能函式 (來自 緣分test.py)
# =================================================================

# -------- 1. 生日 → 星座 --------
def get_zodiac_sign(birthday_str):
    if not birthday_str: return None
    text = birthday_str.strip().replace("/", "-")
    try: d = datetime.datetime.strptime(text, "%Y-%m-%d")
    except ValueError: return None
    m, day = d.month, d.day
    zodiac_ranges = [
        ((3,21), (4,19), "牡羊座"), ((4,20), (5,20), "金牛座"),
        ((5,21), (6,21), "雙子座"), ((6,22), (7,22), "巨蟹座"),
        ((7,23), (8,22), "獅子座"), ((8,23), (9,22), "處女座"),
        ((9,23), (10,23), "天秤座"), ((10,24), (11,22), "天蠍座"),
        ((11,23), (12,21), "射手座"), ((12,22), (12,31), "魔羯座"),
        ((1,1), (1,19), "魔羯座"), ((1,20), (2,18), "水瓶座"),
        ((2,19), (3,20), "雙魚座"),
    ]
    for (sm, sd), (em, ed), name in zodiac_ranges:
        if (m > sm or (m == sm and day >= sd)) and (m < em or (m == em and day <= ed)):
            return name
    return None

# -------- 2. 星座資料 API（Horostory Planetary Overview）--------
HORO_CACHE = {"date": None, "data": "", "fallback": True}
def get_planetary_overview():
    today = datetime.datetime.now().strftime("%Y-%m-%d")
    if HORO_CACHE["date"] == today:
        print(f"📦 使用 Horostory 今日快取（{today}）")
        return HORO_CACHE["data"], HORO_CACHE["fallback"]
    print("🌐 呼叫 Horostory API 取得今日星象...")
    url = "https://horostory.p.rapidapi.com/planetaryoverview"
    headers = {
        "x-rapidapi-key": RAPIDAPI_KEY,
        "x-rapidapi-host": "horostory.p.rapidapi.com"
    }
    try:
        resp = requests.get(url, headers=headers, timeout=12)
        if resp.status_code != 200:
            HORO_CACHE.update({"date": today, "data": "", "fallback": True})
            return "", True
        data = resp.json()
        text = json.dumps(data, ensure_ascii=False)
        HORO_CACHE.update({"date": today, "data": text[:2000], "fallback": False})
        print("🔮 今日星象取得成功（已快取）")
        return HORO_CACHE["data"], False
    except Exception as e:
        print(f"❌ Horostory API 連線或 timeout 錯誤 → fallback：{e}")
        HORO_CACHE.update({"date": today, "data": "", "fallback": True})
        return "", True

# -------- 3. Love Calculator API --------
def get_love_calculator(name1, name2):
    if not RAPIDAPI_KEY or "👉" in RAPIDAPI_KEY: return 50, "尚未設定 RapidAPI Key，使用預設值 50 分"
    url = "https://love-calculator.p.rapidapi.com/LoveCalculator/calculate"
    headers = {
        "x-rapidapi-key": RAPIDAPI_KEY,
        "x-rapidapi-host": "love-calculator.p.rapidapi.com",
        "X-API-KEY": "de305d54-75b4-431b-adb2-eb6b9e546014",
    }
    params = {"FirstName": name1, "SecondName": name2}
    try:
        resp = requests.get(url, headers=headers, params=params, timeout=10)
        resp.raise_for_status()
        data = resp.json()
        percentage = int(data.get("percentage", 50))
        result = data.get("result", "")
        return percentage, result
    except Exception as e:
        print("Love Calculator API 錯誤：", e)
        return 50, "呼叫 love calculator 失敗，使用預設值 50 分"

# -------- 4. 興趣文字 → set & 重疊度 --------
def parse_list_text(text):
    if not text: return set()
    t = text
    for sep in ["、", "，", ",", "/", ";", "｜", "|"]:
        t = t.replace(sep, ",")
    items = [x.strip() for x in t.split(",") if x.strip()]
    return set(items)

def calc_overlap(set1, set2):
    if not set1 or not set2: return 0, 0, 0.0
    inter = set1 & set2
    union = set1 | set2
    if not union: return 0, 0, 0.0
    ratio = len(inter) / len(union)
    return len(inter), len(union), ratio

# -------- 5. Gemini 綜合分析：個性互補度 + 個人化戀愛建議 --------
def gemini_analysis(name1, zodiac1, mbti1, interests1, traits1,
                    name2, zodiac2, mbti2, interests2, traits2,
                    love_score, love_text, interest_overlap_percent,
                    planetary_raw):
    if 'gemini_model' not in globals() or gemini_model is None:
        msg = "⚠️ 尚未設定 Gemini API Key，因此無法產生 AI 分析。"
        return msg, msg

    planetary_short = (planetary_raw or "")[:1500]
    prompt = f"""
你是一位溫柔的感情諮商師，會用淺白、有同理心的中文說明，並給予情緒價值。
請根據以下兩個人的資料，分析他們的關係，並給出具體建議。
... (省略長 prompt 以節省篇幅) ...
【個性互補度】
（用 2~3 段話，以與對象A對話的方式，綜合星座、MBTI、興趣與人格，描述兩人的互補與可能衝突點。）

【個人化戀愛建議】
1. ...
2. ...
3. ...
"""
    try:
        # 重建完整的 prompt 以確保 AI 輸出正確
        full_prompt = f"""
你是一位溫柔的感情諮商師，會用淺白、有同理心的中文說明，並給予情緒價值。

請根據以下兩個人的資料，分析他們的關係，並給出具體建議。

【對象 A】
- 姓名：{name1}
- 星座：{zodiac1 or "未知"}
- MBTI：{mbti1 or "未填"}
- 興趣：{interests1 or "未填"}
- 人格特質：{traits1 or "未填"}

【對象 B】
- 姓名：{name2}
- 星座：{zodiac2 or "未知"}
- MBTI：{mbti2 or "未填"}
- 興趣：{interests2 or "未填"}
- 人格特質：{traits2 or "未填"}

【客觀數據】
- Love Calculator 分數：{love_score} 分，描述：{love_text}
- 興趣重疊度：約 {interest_overlap_percent}%
- 今日星象原始資料（僅供你參考，不用逐條解釋給使用者）：{planetary_short}

請用以下格式輸出（不要加其他說明）：

【個性互補度】
（用 2~3 段話，以與對象A對話的方式，綜合星座、MBTI、興趣與人格，描述兩人的互補與可能衝突點。）

【個人化戀愛建議】
1. ...
2. ...
3. ...
"""

        res = gemini_model.generate_content([full_prompt])
        text = getattr(res, "text", "").strip()
        if not text:
            return "⚠️ Gemini 沒有回傳文字內容，可能是配額或權限問題。", "⚠️ Gemini 沒有回傳文字內容，可能是配額或權限問題。"

        if "【個人化戀愛建議】" in text:
            p1, p2 = text.split("【個人化戀愛建議】", 1)
            personality = p1.replace("【個性互補度】", "").strip()
            advice = p2.strip()
        else:
            personality = text
            advice = text
        return personality, advice
    except Exception as e:
        print("❌ Gemini 產生內容時發生錯誤：", e)
        traceback.print_exc()
        msg = f"⚠️ Gemini 產生內容時發生錯誤：{e}"
        return msg, msg

# -------- 6. 寫入 Google Sheet --------
def ensure_sheet_header():
    expected = [
        "日期", "A_姓名", "A_生日", "A_星座", "A_興趣", "A_人格特質", "A_MBTI",
        "B_姓名", "B_生日", "B_星座", "B_興趣", "B_人格特質", "B_MBTI",
        "LoveCalc分數", "興趣重疊度(%)", "緣分指數", "個性互補度", "戀愛建議","Love與星象分析"
    ]
    try:
        header = ws.row_values(1)
        if header != expected:
            ws.delete_rows(1)
            ws.insert_row(expected, 1)
            print("✅ 已自動建立標題列")
        else:
            print("✔️ 標題列已存在，略過建立")
    except Exception as e:
         print(f"❌ 檢查/建立標題列失敗：{e} (可能無法連線到 Google Sheet)")

def append_to_sheet(row):
    if 'ws' not in globals() or ws is None:
        return
    try:
        ensure_sheet_header()
        ws.append_row(row, value_input_option="USER_ENTERED")
        print("✅ 已寫入試算表一筆紀錄")
    except Exception as e:
        print("❌ 寫入試算表失敗：", e)


# -------- 7. Love Calculator + 星象 / 星座 綜合分析 --------
def gemini_love_astro_analysis(
    name1, name2, zodiac1, zodiac2, love_score, love_text,
    interest_overlap_percent, planetary_raw, horo_fallback: bool
):
    if 'gemini_model' not in globals() or gemini_model is None:
        return "尚未設定 Gemini API Key，因此無法分析 Love Calculator 與星象 / 星座資料。"

    planetary_short = (planetary_raw or "")[:1500]

    # 這裡的 Prompt 會根據 horo_fallback 變動 (省略長 prompt 以節省篇幅)
    if horo_fallback:
        prompt = f"""
你是一位理性、溫和的感情顧問。
...
【Love Calculator】
- 分數：{love_score}
- 系統說明：{love_text}
- 興趣重疊度：約 {interest_overlap_percent}%
...
"""
    else:
        prompt = f"""
你是一位理性、溫和的感情占星顧問，專門解讀「Love Calculator + 星象運勢」。
...
【Love Calculator】
- 分數：{love_score}
- 系統說明：{love_text}

【興趣重疊度】
- 約 {interest_overlap_percent}%

【Horostory 星象 JSON（僅供你提取與人際 / 感情相關部分，請勿逐字輸出）】
{planetary_short}
...
"""

    try:
        res = gemini_model.generate_content([prompt])
        text = getattr(res, "text", "").strip()
        return text if text else "Gemini 未回傳內容，請稍後再試。"
    except Exception as e:
        print("❌ Gemini Love+星象解析錯誤：", e)
        traceback.print_exc()
        return f"Gemini 無法分析 Love Calculator 與星象 / 星座資料：{e}"


# -------- 8. 主配對邏輯函式 --------
def match_pair(name1, birthday1, interests1, traits1, mbti1,
               name2, birthday2, interests2, traits2, mbti2):
    try:
        if not name1 or not birthday1 or not name2 or not birthday2:
            return "❗請至少填寫雙方的『姓名』與『生日』（YYYY-MM-DD）", "", "", ""
        zodiac1 = get_zodiac_sign(birthday1)
        zodiac2 = get_zodiac_sign(birthday2)
        if zodiac1 is None or zodiac2 is None:
            return "❗生日格式請使用 YYYY-MM-DD（例如：2000-05-21）", "", "", ""

        love_score_raw, love_text = get_love_calculator(name1, name2)
        inter_set1 = parse_list_text(interests1)
        inter_set2 = parse_list_text(interests2)
        inter_n, union_n, overlap_ratio = calc_overlap(inter_set1, inter_set2)
        interest_overlap_percent = int(round(overlap_ratio * 100)) if union_n > 0 else 0
        planetary_raw, horo_fallback = get_planetary_overview()
        fate_score = int(round(0.7 * love_score_raw + 0.3 * interest_overlap_percent))

        personality_text, advice_text = gemini_analysis(
            name1, zodiac1, mbti1, interests1, traits1, name2, zodiac2,
            mbti2, interests2, traits2, love_score_raw, love_text,
            interest_overlap_percent, planetary_raw
        )
        love_astro_text = gemini_love_astro_analysis(
            name1, name2, zodiac1, zodiac2, love_score_raw, love_text,
            interest_overlap_percent, planetary_raw, horo_fallback
        )

        summary = (
            f"緣分指數：{fate_score} 分\n"
            f"Love Calculator 原始分數：{love_score_raw} 分\n"
            f"星座：{name1} 是 {zodiac1}，{name2} 是 {zodiac2}\n"
            f"興趣重疊度：約 {interest_overlap_percent}% "
            f"(共有 {inter_n} 項重疊 / 合計 {union_n} 項)\n"
        )
        timestamp = datetime.datetime.now().strftime("%Y-%m-%d")
        row = [
            timestamp, name1, birthday1, zodiac1, interests1, traits1, mbti1,
            name2, birthday2, zodiac2, interests2, traits2, mbti2,
            love_score_raw, interest_overlap_percent, fate_score,
            personality_text, advice_text, love_astro_text
        ]
        append_to_sheet(row)
        return str(summary), str(personality_text), str(advice_text), str(love_astro_text)

    except Exception as e:
        print("❌ match_pair 發生錯誤：", e)
        traceback.print_exc()
        msg = f"系統發生錯誤：{e}"
        return msg, "", "", ""

# =================================================================
# 月老籤詩功能函式 (來自 月老籤詩_正式版.py)
# =================================================================

try:
    df = pd.read_csv("lots.csv", dtype=str).dropna(subset=["original"]).fillna("").reset_index(drop=True)
    print("✅ lots.csv 載入成功。")
except Exception as e:
    print(f"❌ lots.csv 載入失敗：{e}。請確認檔案已上傳。")
    df = pd.DataFrame({"lot_id": ["0"], "original": ["無法載入籤詩資料"], "base_mean": ["無法載入原始解說"], "source_url": [""]})

def draw_lot():
    if df.empty or df.shape[0] == 0:
        return {"lot_id": "0", "original": "籤詩資料庫空白", "base_mean": "請檢查 lots.csv 檔案", "source_url": ""}
    row = df.sample(1).iloc[0]
    return {"lot_id": row["lot_id"], "original": row["original"], "base_mean": row["base_mean"], "source_url": row["source_url"]}

def interpret_lot(lot_data):
    # 使用全域變數 yuelao_client (在 Cell 3 設定)
    if 'yuelao_client' not in globals() or yuelao_client is None:
         return "月老 AI Client 未設定，無法解籤。"
    prompt = f"""
你是一位現代的月老 AI。
以下是使用者抽到的籤詩，請給：
1. 白話翻譯（約 2～3 句）
2. 感情狀態判斷（例如：適合主動、適合等待、先觀察對方）
3. 今日行動建議（一句話）
4. 感情小語（一句短句）

--- 籤詩資料 ---
籤號：{lot_data['lot_id']}
籤文：{lot_data['original']}
原始解說：{lot_data['base_mean']}
"""
    try:
        response = yuelao_client.models.generate_content(
            model="gemini-2.5-flash",
            contents=prompt
        )
        return response.text
    except Exception as e:
        print("❌ Gemini 解籤錯誤：", e)
        return f"月老 AI 解籤發生錯誤：{e}"

def get_fortune_result():
    lot = draw_lot()
    ai_text = interpret_lot(lot)
    return {
        "lot_id": lot["lot_id"], "original": lot["original"],
        "base_mean": lot["base_mean"], "ai_interpretation": ai_text,
        "source_url": lot["source_url"]
    }

# 月老籤詩 Gradio 介面邏輯 (原程式碼不變)
def draw_or_next_ui(state):
    if not state:
        result = get_fortune_result()
        return ("", "", result, gr.update(visible=True), gr.update(visible=False), gr.update(visible=False), gr.update(visible=True))
    lot_title = f"第 {state['lot_id']} 籤"
    poem_html = "<br>".join(str(state["original"]).splitlines())
    return (lot_title, poem_html, state, gr.update(visible=False), gr.update(visible=True), gr.update(visible=False), gr.update(visible=False))

def reveal_lot_ui(state):
    if not state:
        return ("請先從籤筒抽一支籤喔～", "", gr.update(visible=True), gr.update(visible=False), gr.update(visible=False))
    return (state["base_mean"], state["ai_interpretation"], gr.update(visible=False), gr.update(visible=False), gr.update(visible=True))

def reset_ui():
    return ("", "", "", "", None, gr.update(visible=True), gr.update(visible=False), gr.update(visible=False), gr.update(visible=False))

print("✅ 所有核心功能函式定義完成。")

✅ lots.csv 載入成功。
✅ 所有核心功能函式定義完成。


In [ ]:
# -*- coding: utf-8 -*-
# Cell 5：Gradio 介面定義與執行

# 沿用月老籤詩的 CSS 樣式
custom_css = """
/* 整體背景：粉膚色 */
body, .gradio-container {
  background: #e3b39b;
}

/* 中央卡片：米白色 + 紅邊框 */
.card {
  background: #fff6e6;
  border-radius: 28px;
  border: 2px solid #c86b42;
  padding: 24px 32px;
  max-width: 900px;
  margin: 24px auto 40px auto;
  box-shadow: 0 6px 18px rgba(0,0,0,0.08);
}

/* 標題文字：磚紅色 */
#title h1, #title h2, #title h3, #title p {
  color: #9a3b24;
}

/* 說明文字顏色 */
p, label, .gradio-container {
  color: #7a3a28;
}

/* 籤筒圖片 */
.qiantong img {
  max-height: 260px;
  margin: 0 auto 8px auto;
  display: block;
}

/* 主按鈕（抽籤、解籤、再抽一次）：磚紅底、白字、圓角、粗體 */
.primary-btn {
  background: #b5482b !important;    /* 磚紅色 */
  color: #ffffff !important;         /* 白字 */
  border-radius: 999px !important;   /* 超圓 */
  border: none !important;
  font-size: 18px !important;
  font-weight: 700 !important;
  padding: 12px 32px !important;
}

/* 主按鈕 hover：小小變亮 */
.primary-btn:hover {
  filter: brightness(1.08);
}


/* Textbox 邊框、背景 */
textarea, input {
  background: #fffdf7 !important;
  border-radius: 16px !important;
  border: 1px solid #e1b08f !important;
}

/* Textbox 標題顏色 */
label {
  font-weight: 600;
  color: #9a3b24;
}

/* 籤號：置中、放大、磚紅色 */
.lot-title textarea,
.lot-title input {
  text-align: center;         /* 文字置中 */
  font-size: 22px;
  font-weight: 700;
  color: #b5482b;             /* 接近雷雨師一百籤的紅色 */
}

/* 籤詩原文外框 */
.poem-box {
  background-color: #fffdf7;
  border-radius: 24px;
  border: 2px solid #d9a679;
  height: 300px;
  display: flex;
  align-items: center;
  justify-content: center;
  margin: 0 auto;
  background-image:
    radial-gradient(circle at 20% 20%, rgba(255, 204, 170, 0.25), transparent 60%),
    radial-gradient(circle at 80% 70%, rgba(255, 220, 190, 0.22), transparent 65%);
  background-blend-mode: multiply;
}


/* 籤詩原文字：直式、從右到左、粗體、放大、無陰影 */
.poem-box > div {
  writing-mode: vertical-rl;
  text-orientation: upright;
  font-size: 32px;
  font-weight: 900;
  line-height: 2.0;
  letter-spacing: 0.18em;
  color: #b5482b;
}

/* Tab 標籤顏色 */
.gradio-tabs .tab-nav button.selected {
  background-color: #9a3b24 !important; /* 選中的 Tab 背景 */
  color: #fff !important; /* 選中的 Tab 文字 */
  font-weight: 700;
}
.gradio-tabs .tab-nav button {
  color: #9a3b24; /* 未選中的 Tab 文字 */
}
"""


with gr.Blocks(title="💘月老緣分分析系統", css=custom_css) as demo:
    gr.Markdown("## 💘月老緣分分析系統", elem_id="title")

    with gr.Tabs():

        # ----------------------------------------
        # Tab 1: 緣分匹配 (第一頁)
        # ----------------------------------------
        with gr.TabItem("緣分匹配"):
            gr.Markdown("請輸入雙方基本資料，我將算出你們的緣分、個性互補和戀愛建議。")

            with gr.Column(elem_classes=["card"]):
                with gr.Row():
                    with gr.Column():
                        gr.Markdown("### 對象 A (你)")
                        name1 = gr.Textbox(label="姓名（建議英文）", placeholder="ex: Alice")
                        birthday1 = gr.Textbox(label="生日（YYYY-MM-DD）", placeholder="ex: 2002-05-21")
                        interests1 = gr.Textbox(label="興趣", placeholder="多項請用逗號或頓號，如：音樂、閱讀、運動")
                        traits1 = gr.Textbox(label="人格特質", placeholder="ex: 外向、溫柔、有耐心")
                        mbti1 = gr.Textbox(label="MBTI（選填）", placeholder="ex: INFP / ESTJ")

                    with gr.Column():
                        gr.Markdown("### 對象 B (他)")
                        name2 = gr.Textbox(label="姓名（建議英文）", placeholder="ex: Jeon jung kook")
                        birthday2 = gr.Textbox(label="生日（YYYY-MM-DD）", placeholder="ex: 1997-09-01")
                        interests2 = gr.Textbox(label="興趣", placeholder="多項請用逗號或頓號，如：電影、烹飪、旅遊")
                        traits2 = gr.Textbox(label="人格特質", placeholder="ex: 調皮、理性、善解人意")
                        mbti2 = gr.Textbox(label="MBTI（選填）", placeholder="ex: INTP")

                btn = gr.Button("開始配對！", elem_classes=["primary-btn"])

                fate_output = gr.Textbox(label="綜合緣分指數", lines=5, interactive=False)
                personality_output = gr.Textbox(label="個性互補度（AI 分析）", lines=6, interactive=False)
                advice_output = gr.Textbox(label="個人化戀愛建議", lines=8, interactive=False)
                love_astro_output = gr.Textbox(label="Love Calculator ＆ 星象解析", lines=8, interactive=False)

                btn.click(
                    fn=match_pair,
                    inputs=[name1, birthday1, interests1, traits1, mbti1,
                            name2, birthday2, interests2, traits2, mbti2],
                    outputs=[fate_output, personality_output, advice_output, love_astro_output]
                )

        # ----------------------------------------
        # Tab 2: 月老籤詩 (第二頁)
        # ----------------------------------------
        with gr.TabItem("月老籤詩"):
            yuelao_state = gr.State()

            with gr.Column(elem_classes=["card"]):
                gr.Markdown("靜一下，想著你現在的心念，從籤筒抽出一支屬於你的籤詩。")

                # 🔸 畫面一：只有籤筒 + 抽籤按鈕
                with gr.Group(visible=True, elem_classes=["screen-block"]) as screen0:
                    with gr.Column(elem_id="main-col"):
                        # ⚠️ 請在 Colab 檔案列表上傳 qiantong.png
                        gr.Image(
                            value="qiantong.png",
                            show_label=False,
                            interactive=False,
                            elem_classes=["qiantong"],
                        )
                        draw_btn = gr.Button("抽一支月老籤", elem_classes=["primary-btn"])
                        loading_text = gr.Markdown("正在抽出屬於你的緣分籤詩…", visible=False)

                # 🔸 畫面二：籤號 + 籤詩原文 + 解籤按鈕
                with gr.Group(visible=False, elem_classes=["screen-block"]) as screen1:
                    lot_title = gr.Textbox(label="籤號", interactive=False, elem_classes=["lot-title"])
                    gr.Markdown("籤詩原文")
                    original = gr.HTML(value="", elem_classes=["poem-box"])
                    reveal_btn = gr.Button("解籤", elem_classes=["primary-btn"])

                # 🔸 畫面三：詳細解籤結果
                with gr.Group(visible=False, elem_classes=["screen-block"]) as screen2:
                    base_mean = gr.Textbox(label="原始解釋", lines=5, interactive=False)
                    ai_text = gr.Textbox(label="月老解籤", lines=10, interactive=False)
                    back_btn   = gr.Button("再抽一次", elem_classes=["primary-btn"])

            # 綁定邏輯
            draw_btn.click(
                fn=draw_or_next_ui,
                inputs=yuelao_state,
                outputs=[lot_title, original, yuelao_state, screen0, screen1, screen2, loading_text],
            )

            reveal_btn.click(
                fn=reveal_lot_ui,
                inputs=yuelao_state,
                outputs=[base_mean, ai_text, screen0, screen1, screen2],
            )

            back_btn.click(
                fn=reset_ui,
                inputs=None,
                outputs=[lot_title, original, base_mean, ai_text, yuelao_state, screen0, screen1, screen2, loading_text],
            )

    demo.launch(share=True, debug=True)

/tmp/ipython-input-359003424.py:118: DeprecationWarning: The 'css' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'css' to Blocks.launch() instead.
  with gr.Blocks(title="💘月老緣分分析系統", css=custom_css) as demo:


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://da0151c6fbd8642e34.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
